<a href="https://colab.research.google.com/github/vagant50/DLFS_code/blob/master/%D0%A0%D0%B0%D0%B7%D0%B1%D0%BE%D1%80_%D0%94%D0%97_%D0%BA_%D0%BE%D1%82%D0%BA%D1%80%D1%8B%D1%82%D0%BE%D0%BC%D1%83_%D1%83%D1%80%D0%BE%D0%BA%D1%83_%D0%BF%D0%BE_%D0%B4%D0%BE%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8E_ChatGPT_%7C_%D0%A3%D0%98%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

На основе базы знаний компании "Островок" сделайте нейро-консультанта для ответов на вопросы по бронированиям. Подставьте следующие документы в код, как это было сделано в учебных материалах
Документ с базой знаний можно найти тут: https://docs.google.com/document/d/1K8kIpWxUX8GNBtmPnStYy83zpV8zfVeBqWVVlH7wNdU
Промпт можно найти тут: https://docs.google.com/document/d/1JCp4OWP22EZTU-HllyanhYfNSy_RwJ1aP7dnXPY2hqE


In [ ]:
!pip install openai
!pip -q install --upgrade tiktoken
!pip -q install langchain openai chromadb

from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
import re
# игнорирование предупреждений
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("langchain.text_splitter").setLevel(logging.ERROR)
logging.getLogger("chromadb").setLevel(logging.ERROR)
from getpass import getpass

import openai
import tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing 

In [ ]:
# Запрашивает у пользователя ключ API и сохраняет его в переменную
api_key = getpass('Введите ваш ключ API:')

# Устанавливаем ключ API напрямую через openai.api_key
openai.api_key = api_key


Введите ваш ключ API:··········


In [ ]:
def load_document_text(url: str) -> str:
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

def create_search_index(text: str) -> Chroma:
    return create_embedding(text)


def create_embedding(data):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
        """Returns the number of tokens in a text string."""
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens

    source_chunks = []
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

    for chunk in splitter.split_text(data):
        source_chunks.append(Document(page_content=chunk, metadata={}))

    # Создание индексов документа
    search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings(openai_api_key=api_key), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")

    return search_index

def answer(system, topic, temp = 1):
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )

    return completion.choices[0].message.content

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):

    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":
        num_tokens = 0
        for message in messages:
            num_tokens += 4
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":
                    num_tokens += -1
        num_tokens += 2
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)


def answer_index(system, topic, search_index, temp=1, verbose=0):

    docs = search_index.similarity_search(topic, k=5)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system + f"{message_content}"},
        {"role": "user", "content": topic}
    ]

    if verbose: print('\n ===========================================: ')
    if verbose: print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} tokens used for the question")

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp
    )

    if verbose: print('\n ===========================================: ')
    if verbose: print(f'{completion["usage"]["total_tokens"]} total tokens used (question-answer).')
    if verbose: print('\n ===========================================: ')
    answer = insert_newlines(completion.choices[0].message.content)
    return answer

def get_chatgpt_ansver3(system, topic, search_index, temp = 1):

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": topic}
    ]

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp
    )
    print('ANSWER : \n', insert_newlines(completion.choices[0].message.content))

def answer_user_question(system_doc: str, knowledge_base_url: str, user_question: str) -> str:

    system_doc_text = load_document_text(system_doc_url)
    knowledge_base_text = load_document_text(knowledge_base_url)

    # Создаем индексы поиска
    knowledge_base_index = create_search_index(knowledge_base_text)

    input_text = user_question

    answer_text = answer_index(system_doc_text, input_text, knowledge_base_index, temp=temperature, verbose=verbose)

    return answer_text

In [ ]:
temperature=0.5
verbose=0

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system_doc_url = 'https://docs.google.com/document/d/1JCp4OWP22EZTU-HllyanhYfNSy_RwJ1aP7dnXPY2hqE'

In [ ]:
# База знаний, которая будет подаваться в LangChain
knowledge_base_url = 'https://docs.google.com/document/d/1K8kIpWxUX8GNBtmPnStYy83zpV8zfVeBqWVVlH7wNdU' # база знаний

In [ ]:
# вопрос пользователя
topic= "Как мне получить документы для предоставления в бухгалтерию моей компании после командировки?"

In [ ]:
ans=answer_user_question(system_doc_url, knowledge_base_url, topic)
ans

' При направлении вас в командировку документами, подтверждающими расходы по оплате билетов и/или услуг гостиницы, являются чеки контрольно-кассовой техники, слипы или\n чеки электронных терминалов, если вы оплачивали услуги банковской картой, которая находится на ваше имя. Другие документы, такие как бумажные командировочные, мы не\n предоставляем. Если вам нужны дополнительные документы или вы хотите бронировать командировки с полным пакетом закрывающих документов, пожалуйста, обратитесь в отдел по\n работе с корпоративными клиентами на почту corporate@ostrovok.ru.'